In [5]:
import numpy as np
import random
import math
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
from queue import Queue
import warnings
import sys
import json
import time
import types
import pickle
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import os

In [7]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [ ]:
class MapGenerator:
    def __init__(
        self,
        map_type: Literal["random", "multi_narrow", "maze"] = "random",
        map_size: Union[Tuple[int, int], Tuple[int, int, int]] = (50, 50),
        obstacle_percent: float = 0.2,
        min_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (2, 2),
        max_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (5, 5),
        max_obstacle_count: Optional[int] = None
    ):
        self.map_type = map_type
        self.map_size = map_size
        self.obstacle_percent = obstacle_percent
        self.obstacle_list: List[Tuple[int, ...]] = []
        self.min_size = min_obstacle_size
        self.max_size = max_obstacle_size
        self.max_count = max_obstacle_count
        self.is_3d = len(map_size) == 3
        self.map = self._init_map()
        self.start = None
        self.goal = None

    def _init_map(self):
        shape = self.map_size[::-1] if self.is_3d else (self.map_size[1], self.map_size[0])
        return np.zeros(shape, dtype=np.uint8)

    def generate(self, start: Tuple[int, ...], goal: Tuple[int, ...]) -> Dict[str, Union[np.ndarray, Tuple[int, ...], List[Tuple[int, ...]], Tuple[int, int]]]:
        self.start = start
        self.goal = goal
        tries = 0
        max_tries = 100

        while tries < max_tries:
            self.map = self._init_map()
            self.obstacle_list.clear()

            if self.map_type == "maze":
                self.map = self._generate_maze()
            elif self.map_type == "random":
                self._generate_random_obstacles(start, goal)
            elif self.map_type == "multi_narrow":
                self._generate_multi_narrow(start, goal)

            if self._path_exists(start, goal):
                return Map(
                    grid=self.map,
                    start=start,
                    goal=goal,
                    obstacles=self.obstacle_list,
                    size=self.map_size
                )
            tries += 1

        raise RuntimeError("Failed to generate a connected map after multiple attempts.")

    def _add_obstacle(self, coords: Tuple[int, ...]):
        self.obstacle_list.append(coords)

    def _path_exists(self, start, goal):
        visited = set()
        q = Queue()
        q.put(start)
        visited.add(start)

        dims = len(start)
        neighbors = [(-1,0), (1,0), (0,-1), (0,1)] if dims == 2 else \
                    [(-1,0,0),(1,0,0),(0,-1,0),(0,1,0),(0,0,-1),(0,0,1)]

        while not q.empty():
            node = q.get()
            if node == goal:
                return True

            for delta in neighbors:
                neighbor = tuple(node[i] + delta[i] for i in range(dims))
                if self._in_bounds(neighbor) and neighbor not in visited:
                    if self._is_free(neighbor):
                        visited.add(neighbor)
                        q.put(neighbor)
        return False

    def _in_bounds(self, p):
        if self.is_3d:
            z, y, x = p
            return 0 <= z < self.map.shape[0] and 0 <= y < self.map.shape[1] and 0 <= x < self.map.shape[2]
        else:
            y, x = p
            return 0 <= y < self.map.shape[0] and 0 <= x < self.map.shape[1]

    def _is_free(self, p):
        if self.is_3d:
            z, y, x = p
            return self.map[z, y, x] == 0
        else:
            y, x = p
            return self.map[y, x] == 0

    def _is_inside(self, point, x, y, z, ow, oh, od):
        px, py, *pz = point
        if z is None:
            return x <= px < x + ow and y <= py < y + oh
        else:
            pz = pz[0]
            return x <= px < x + ow and y <= py < y + oh and z <= pz < z + od

    def _generate_random_obstacles(self, start, goal):
        if self.is_3d:
            w, h, d = self.map_size
            total_voxels = w * h * d
            max_obs = int(total_voxels * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                od = random.randint(self.min_size[2], self.max_size[2])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                z = random.randint(0, d - od - 1)
                if np.any(self.map[z:z+od, y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, z, ow, oh, od) or self._is_inside(goal, x, y, z, ow, oh, od):
                    continue
                self.map[z:z+od, y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, z, ow, oh, od))
                count += ow * oh * od
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break
        else:
            w, h = self.map_size
            total_cells = w * h
            max_obs = int(total_cells * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                if np.any(self.map[y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, None, ow, oh, None) or self._is_inside(goal, x, y, None, ow, oh, None):
                    continue
                self.map[y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, ow, oh))
                count += ow * oh
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break

    def _generate_multi_narrow(self, start, goal):
        if self.is_3d:
            self._generate_random_obstacles(start, goal)
        else:
            w, h = self.map_size
            corridor_width = 2
            spacing = 6
            for i in range(spacing, h - spacing, spacing + corridor_width):
                self.map[i:i+spacing, :] = 1
            self._carve_corridor(start, goal)

    def _carve_corridor(self, start, goal):
        if not self.is_3d:
            y0, x0 = start[1], start[0]
            y1, x1 = goal[1], goal[0]
            for x in range(min(x0, x1), max(x0, x1)+1):
                self.map[y0, x] = 0
            for y in range(min(y0, y1), max(y0, y1)+1):
                self.map[y, x1] = 0

    def _generate_maze(self):
        width, height = self.map_size
        if width % 2 == 0: width += 1
        if height % 2 == 0: height += 1

        maze = np.ones((height, width), dtype=np.uint8)
        sx, sy = 1, 1
        maze[sy, sx] = 0

        walls = [(sx + dx, sy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                 if 0 < sx + dx < width and 0 < sy + dy < height]

        while walls:
            wx, wy = walls.pop(random.randint(0, len(walls) - 1))
            if maze[wy, wx] == 1:
                neighbors = [(wx + dx, wy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                             if 0 < wx + dx < width and 0 < wy + dy < height and maze[wy + dy, wx + dx] == 0]
                if len(neighbors) == 1:
                    nx, ny = neighbors[0]
                    maze[(wy + ny) // 2, (wx + nx) // 2] = 0
                    maze[wy, wx] = 0
                    for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]:
                        nx, ny = wx + dx, wy + dy
                        if 0 < nx < width and 0 < ny < height and maze[ny, nx] == 1:
                            walls.append((nx, ny))

        return maze

In [8]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [4]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [ ]:
gen = MapGenerator(map_type="random", map_size=(100, 100), min_obstacle_size=(5, 5), max_obstacle_size=(15, 15), obstacle_percent=0.32)
m = gen.generate(start=(1, 1), goal=(99, 99))
visualize_map_shapes(m.grid, obs=m.obstacles, start=m.start, goal=m.goal)

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (80,50), (50,50)
    

    grid[27:30, 52:70] = 1  
    obstacles.append((27, 52, 3, 18))
    grid[27:30, 30:48] = 1  
    obstacles.append((27, 30, 3, 18))
    
    grid[70:73, 30:70] = 1
    obstacles.append((70, 30, 3, 40))

    grid[30:70, 30:33] = 1
    obstacles.append((30, 30, 40, 3))
    grid[30:70, 67:70] = 1  
    obstacles.append((30, 67, 40, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,40), (90,10)
    


    grid[17:20, 20:80] = 1
    obstacles.append((17, 20, 3, 60))
    grid[37:40, 0:50] = 1
    obstacles.append((37, 0, 3, 50))
    grid[57:60, 20:100] = 1
    obstacles.append((57, 20, 3, 80))
    grid[77:80, 20:70] = 1
    obstacles.append((77, 0, 3, 80))

    grid[20:37, 47:60] = 1  
    obstacles.append((20, 47, 17, 3))
    grid[30:57, 62:65] = 1 
    obstacles.append((30, 62, 27, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((10, 80, 35, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((70, 80, 20, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [112]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,10), (90,80)
    
    grid[27:30, 20:80] = 1
    obstacles.append((27, 20, 3, 60))
    grid[47:50, 0:50] = 1
    obstacles.append((47, 0, 3, 50))
    grid[77:80, 20:100] = 1
    obstacles.append((77, 20, 3, 80))

    grid[30:47, 47:50] = 1  
    obstacles.append((30, 47, 17, 3))
    grid[50:77, 72:65] = 1 
    obstacles.append((50, 72, 27, 3))

    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [113]:
map_obj = generate_box_with_narrow_entry(width=100, height=100, entry_side="left", entry_pos=25, entry_width=2)


In [4]:
visualize_map_shapes(map_obj.grid, obs=map_obj.obstacles, start=map_obj.start, goal=map_obj.goal)

NameError: name 'visualize_map_shapes' is not defined

In [10]:
# MapIO.save_map(map_obj, "Maze_map_easy.pkl")

In [9]:
maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

Start: (29, 10)
Obstacles: 5
(100, 100)


In [ ]:
maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

In [10]:
narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (80, 50)
Obstacles: 5
(100, 100)


In [11]:
# 불러오기
multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)


In [9]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal)

In [32]:
import pandas as pd
import numpy as np
import random
import time
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any
import plotly.express as px
import math
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult
class MultiMapBenchmarker:
    def __init__(
        self,
        maps: List[np.ndarray],
        name: str = "Algorithm",
        learning_mode = True,
        iter = 10,
        seed = 42,
    ):
        self.set_seed(seed)

        self.maps = maps
        self.name = name
        self.results_df: pd.DataFrame = pd.DataFrame()
        self.iter = iter
        self.learning_mode = learning_mode

        self.time_limit = 5.0
        self.success_limit = 0.8

    def run(self, algorithm) -> pd.DataFrame:
        results = []
        main_start_time = time.time()
        for i, map_ in enumerate(self.maps):
            print(f"Map {i+1}")
            for j in range(self.iter):
                start_time = time.time()
                try:
                    output = algorithm(map_)
                except Exception as e:
                    output = {"path": [], "nodes": [], "n_nodes": 0}
                end_time = time.time()


                if isinstance(output, PlannerResult):
                    path = output.path
                    nodes = output.nodes
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                    path_smoothness = self._compute_path_smoothness(path)
                elif isinstance(output, Dict):
                    path = output['path']
                    nodes = output['nodes']
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                    path_smoothness = self._compute_path_smoothness(path)
                else:
                    return None, None
                
                print(f"Iteration {j+1}: Time taken: {end_time - start_time:.4f} seconds, Success: {success}")

                results.append({
                    "map_id": i,
                    "iter" : j,
                    "algorithm": self.name,
                    "success": success,
                    "time_taken": end_time - start_time,
                    "num_nodes": num_nodes,
                    "path_length": path_length,
                    "path_smoothness": path_smoothness,
                })

                time_taken_avg = np.mean([r["time_taken"] for r in results])
                success_avg = np.mean([r["success"] for r in results]) if len(results) > 6 else 1.0
                if time_taken_avg > self.time_limit:
                    print("Time taken Limit")
                    return None, None
                elif success_avg < self.success_limit:
                    print("Success Rate Limit")
                    return None, None

        print(f"Total time taken for all maps: {time.time() - main_start_time:.4f} seconds")

        self.results_df = pd.DataFrame(results)
        return self.results_df, self.get_avg()

    def _compute_path_length(self, path: List[Tuple[float, ...]]) -> float:
        if not path or len(path) < 2:
            return 0.0
        return sum(np.linalg.norm(np.array(path[i]) - np.array(path[i+1])) for i in range(len(path) - 1))
    
    def _is_path_valid(self, path: List[Tuple[float, ...]], map_) -> bool:
        if not path or len(path) < 2:
            return False
        
        if np.linalg.norm(np.array(path[0]) - np.array(map_.start)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.goal)) > 0.5:
            if np.linalg.norm(np.array(path[0]) - np.array(map_.goal)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.start)) > 0.5:
                return False
            
        is_3d = True if len(map_.size) > 2 else False
        for i in range(len(path) - 1):
            p1 = path[i]
            p2 = path[i + 1]
            if self._is_edge_in_obstacle(p1, p2, map_.obstacles, is_3d):
                return False
        return True
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False
    
    def _compute_path_smoothness(self, path: List[Tuple[float, ...]]) -> float:
        """
        Compute smoothness based on total bending angles (smaller is smoother).
        Returns a value where 1 = perfectly smooth (straight), 0 = very jagged.
        """
        if not path or len(path) < 3:
            return 1.0  # trivially smooth

        total_angle = 0.0
        for i in range(1, len(path) - 1):
            p0 = np.array(path[i - 1])
            p1 = np.array(path[i])
            p2 = np.array(path[i + 1])

            v1 = p0 - p1
            v2 = p2 - p1

            norm_v1 = np.linalg.norm(v1)
            norm_v2 = np.linalg.norm(v2)
            if norm_v1 == 0 or norm_v2 == 0:
                continue  # ignore invalid

            cosine = np.dot(v1, v2) / (norm_v1 * norm_v2)
            cosine = np.clip(cosine, -1.0, 1.0)  # numerical safety
            angle = np.arccos(cosine)
            total_angle += angle

        # Normalize to [0, 1]: smoothness = 1 / (1 + total_bend)
        return 1.0 / (1.0 + total_angle)

    def save_results(self, filename: str):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")
        self.results_df.to_csv(filename, index=False)

    def plot_metrics(self, metric: str = "time_taken"):
        if self.results_df.empty:
            raise RuntimeError("No results to plot. Run benchmark first.")
        if metric not in self.results_df.columns:
            raise ValueError(f"Invalid metric: {metric}")

        fig = px.bar(
            self.results_df,
            x="map_id",
            y=metric,
            color="success",
            title=f"{self.name} - {metric} per map",
            labels={"map_id": "Map ID", metric: metric.replace('_', ' ').title()}
        )
        fig.show()

    def get_avg(self):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")

        classic_summary = self.results_df.groupby('map_id').agg({
            'success': 'mean',           # 성공률
            'time_taken': 'mean',        # 평균 소요 시간
            'num_nodes': 'mean',      # 평균 노드 수
            'path_length': lambda x: x[x > 0].mean(),  # path=0 제외한 평균 경로 길이
            'path_smoothness': 'mean' # 경로 부드러움
        }).rename(columns={
            'success': 'success_rate',
            'time_taken': 'time_avg',
            'num_nodes': 'num_nodes_avg',
            'path_length': 'path_length_avg',
            'path_smoothness': 'smoothness_avg'
        }).reset_index()
        return classic_summary
    
    @staticmethod
    def get_improvement(reference_result:pd.DataFrame, results:pd.DataFrame) -> pd.DataFrame:
        improvement_df = pd.DataFrame()
        
        improvement_df['success_improvement'] = (results['success_rate'] - reference_result['success_rate']) * 100 # percent point
        improvement_df['time_improvement'] = (results['time_avg'] - reference_result['time_avg']) / reference_result['time_avg'] * -100
        improvement_df['length_improvement'] = (results['path_length_avg'] - reference_result['path_length_avg']) / reference_result['path_length_avg'] * -100
        improvement_df['smoothness_improvement'] = (results['smoothness_avg'] - reference_result['smoothness_avg']) / reference_result['smoothness_avg'] * -100

        improvement_df['objective_score'] = (
            5 * improvement_df['success_improvement'] +
            0.3 * improvement_df['time_improvement'] +
            0.2 * improvement_df['length_improvement'] +
            0.2 * improvement_df['smoothness_improvement']
            )

        return improvement_df
    
    def set_seed(self, seed: int = 42):
        random.seed(seed)                    # Python random
        np.random.seed(seed)                 # NumPy
        # torch.manual_seed(seed)              # PyTorch (CPU)
        # torch.cuda.manual_seed(seed)         # PyTorch (GPU)
        # torch.cuda.manual_seed_all(seed)     # If multiple GPUs
        # torch.backends.cudnn.deterministic = True  # CUDNN 고정
        # torch.backends.cudnn.benchmark = False     # 연산 최적화 OFF

## Generate templete

In [ ]:
# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections


# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000):
        self.max_iter = max_iter

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

In [15]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'

In [282]:
result = {
    "algorithm": "Improved-RRT*-Connect",
    "algorithm_description": "Improved RRT*-Connect is a bidirectional, asymptotically optimal planner that enhances RRT*-Connect by incorporating informed heuristic sampling, adaptive step size, node rejection, and pruning. It accelerates convergence and improves success rate in narrow, obstacle-rich environments.",
    "planning_mechanism": "The planner grows two trees from start and goal using informed sampling. During expansion, it adaptively adjusts the step size near obstacles, rejects inefficient new nodes, and prunes branches that cannot contribute to an improved solution. The planner rewires nearby nodes only if doing so reduces path cost, and updates the current best path whenever a successful connection is found.",
    "code": '''
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

    def path_from_root(self):
        node, path = self, []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


class Planner:
    def __init__(self, max_iter=5000, step_size=5.0, rewire_radius=15.0):
        self.max_iter = max_iter
        self.base_step = step_size
        self.rewire_radius = rewire_radius

    def plan(self, map):
        import math, random, numpy as np

        bounds = map.size
        start, goal = map.start, map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3
        dim = len(bounds)

        tree_a, tree_b = [Node(start)], [Node(goal)]
        nodes = [tree_a[0], tree_b[0]]
        edges = []
        success, c_best, best_path = False, float("inf"), []
        c_min = math.dist(start, goal)

        for i in range(self.max_iter):
            tree_a, tree_b = (tree_a, tree_b) if i % 2 == 0 else (tree_b, tree_a)

            sample = self._informed_sample(start, goal, c_best, c_min, bounds, dim)
            nearest = min(tree_a, key=lambda n: math.dist(n.position, sample))

            step = self._adaptive_step(nearest.position, sample, obstacles, is_3d)
            new_pos = self._steer(nearest.position, sample, step)

            if self._is_in_obstacle(new_pos, obstacles, is_3d):
                continue
            if self._is_edge_in_obstacle(nearest.position, new_pos, obstacles, is_3d):
                continue

            cost = nearest.cost + math.dist(nearest.position, new_pos)
            if cost + math.dist(new_pos, goal) >= c_best:
                continue  # pruning

            new_node = Node(new_pos, nearest, cost)
            nearest.add_child(new_node)
            tree_a.append(new_node)
            nodes.append(new_node)
            edges.append((nearest, new_node))

            near_nodes = [n for n in tree_a if math.dist(n.position, new_node.position) <= self.rewire_radius]
            for near in near_nodes:
                new_cost = new_node.cost + math.dist(new_node.position, near.position)
                if new_cost < near.cost and not self._is_edge_in_obstacle(new_node.position, near.position, obstacles, is_3d):
                    if near.parent:
                        near.parent.children.remove(near)
                        edges.remove((near.parent, near))
                    near.parent = new_node
                    near.cost = new_cost
                    new_node.add_child(near)
                    edges.append((new_node, near))

            # Try to connect to the other tree
            other_nearest = min(tree_b, key=lambda n: math.dist(n.position, new_node.position))
            connect_cost = new_node.cost + math.dist(new_node.position, other_nearest.position) + other_nearest.cost
            if connect_cost < c_best and not self._is_edge_in_obstacle(new_node.position, other_nearest.position, obstacles, is_3d):
                c_best = connect_cost
                path_a = new_node.path_from_root()
                path_b = other_nearest.path_from_root()
                best_path = path_a + path_b[::-1]
                success = True

        return PlannerResult(success=success, path=best_path, nodes=nodes, edges=edges)

    def _informed_sample(self, start, goal, c_best, c_min, bounds, dim):
        import numpy as np, math, random
        if c_best == float("inf"):
            return tuple(random.uniform(0, bounds[d]) for d in range(dim))
        x_center = [(s + g) / 2 for s, g in zip(start, goal)]
        a1 = np.array(goal) - np.array(start)
        a1 = a1 / np.linalg.norm(a1)
        M = np.outer(a1, np.eye(dim)[0])
        U, _, Vt = np.linalg.svd(M)
        C = U @ np.diag([1] * (dim - 1) + [np.linalg.det(U) * np.linalg.det(Vt)]) @ Vt
        r1 = c_best / 2
        r2 = math.sqrt(c_best**2 - c_min**2) / 2
        L = np.diag([r1] + [r2] * (dim - 1))
        while True:
            x_ball = np.random.normal(0, 1, dim)
            x_ball /= np.linalg.norm(x_ball)
            x_ball *= random.random() ** (1 / dim)
            x_rand = C @ L @ x_ball + x_center
            if all(0 <= x_rand[d] <= bounds[d] for d in range(dim)):
                return tuple(x_rand)

    def _adaptive_step(self, from_pos, to_pos, obstacles, is_3d):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(2, int(distance))
        for i in range(1, steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return max(self.base_step * 0.3, 1.0)
        return self.base_step

    def _steer(self, from_pos, to_pos, step):
        import math
        dist = math.dist(from_pos, to_pos)
        if dist <= step:
            return to_pos
        return tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * step / dist for d in range(len(from_pos)))

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

    '''
}

# 저장
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)
    f.write(",\n")

In [12]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [ ]:
Instruction : Generate Improved-RRT*-Connect algorithm

Refer to below architecture:
!!!!!generate Only class Node and Planner!!!!!!
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float=5.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

### Objective:
- Improve path planning performance in terms of:
  - Planning efficiency
  - Path quality
  - Robustness
  - Success rate
  - Path smoothness
  - Path lengths
  - Reduce search time

### Constraints:
- Implement it in Python.
- You DO NOT NEED to declare the any imports.
- When connecting nodes and adding edges in the planner, always perform two critical checks:
1.Collision check for the node position: Ensure that the new node itself does not lie inside any obstacle.
2.Edge-obstacle intersection check: Before adding an edge between two nodes, verify that the straight-line path between them does not intersect or pass through any obstacle.
- DO NOT OVER MAP BOUND
- After code generation, you must review the code to ensure it is syntactically correct, logically coherent, and executable within the expected environment.
- At the top of your response, write an description of the algorithm in curly braces {}, followed by a concise explanation of the planning mechanism in angle brackets <>.
- Both the description and the planning mechanism should be placed outside and above the code block.
- Output the code block containing the implementation only.
⚠️ Do not give additional explanations.


In [32]:
filtered_sorted_algorithms[0].keys()

dict_keys(['operator', 'algorithm_description', 'planning_mechanism', 'code', 'objective', 'time_improvement', 'length_improvement', 'other_inf', 'success_rate'])

In [42]:
result = filtered_sorted_algorithms[3]

code_string = result['code']
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+code_string, planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module
    p = planning_module.Planner(max_iter=5000, step_size=5.0)


In [43]:
p = Planner(max_iter=5000, step_size=5)
# p = planning_module.Planner()

NameError: name 'Planner' is not defined

In [44]:
result = p.plan(map=multi_obs_map)

In [45]:
result2 = p.plan(map=maze_map)

In [46]:
result3 = p.plan(map=narrow_map)

In [47]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal,
                      path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [48]:
visualize_map_shapes(maze_map.grid, obs=maze_map.obstacles, start=maze_map.start, goal=maze_map.goal, path=result2.path, nodes=list(map(lambda x: x.position, result2.nodes)), edges=result2.edges)

In [49]:
visualize_map_shapes(narrow_map.grid, obs=narrow_map.obstacles, start=narrow_map.start, goal=narrow_map.goal, path=result3.path, nodes=list(map(lambda x: x.position, result3.nodes)), edges=result3.edges)

In [306]:
bench2 = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    name="MyRRT"
)
_, avg_res = bench2.run(p.plan)

Map 1, Iteration 1: Time taken: 4.5771 seconds
Map 1, Iteration 2: Time taken: 4.7441 seconds
Map 1, Iteration 3: Time taken: 4.6237 seconds
Map 1, Iteration 4: Time taken: 4.5852 seconds
Map 1, Iteration 5: Time taken: 3.8997 seconds
Map 1, Iteration 6: Time taken: 4.4552 seconds
Map 1, Iteration 7: Time taken: 4.7395 seconds
Map 1, Iteration 8: Time taken: 4.6186 seconds
Map 1, Iteration 9: Time taken: 4.5787 seconds
Map 1, Iteration 10: Time taken: 4.4073 seconds
Map 2, Iteration 1: Time taken: 4.9983 seconds
Map 2, Iteration 2: Time taken: 5.8869 seconds
Map 2, Iteration 3: Time taken: 6.7470 seconds
Map 2, Iteration 4: Time taken: 6.2999 seconds
Map 2, Iteration 5: Time taken: 5.4660 seconds
Map 2, Iteration 6: Time taken: 6.6855 seconds
Map 2, Iteration 7: Time taken: 6.0104 seconds
Map 2, Iteration 8: Time taken: 5.8618 seconds
Map 2, Iteration 9: Time taken: 5.9092 seconds
Map 2, Iteration 10: Time taken: 7.0804 seconds
Map 3, Iteration 1: Time taken: 8.1677 seconds
Map 3, Iter

In [ ]:
def my_dummy_planner(map_array, start, goal):
    return {
        "path": [start, goal],
        "visited": [start],
        "nodes": 2
    }

bench = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    algorithm=p.plan,
    name="MyRRT"
)


df = bench.run()
print(df)

In [12]:
import_string ='''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map

'''

In [11]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [15]:
classic_method[0]['algorithm']

'RRT'

In [13]:
for method in classic_method:
    if method['algorithm'] == 'RRT':
        print(method['code'])


class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child_node):
        self.children.append(child_node)


class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 3.0, goal_sample_rate: float = 0.05):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate

    def plan(self, map) -> PlannerResult:
        import random
        import math

        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        success_state = False
        extracted_path = []
        nodes = []
        edges = []

        root = Node(start_position)
        nodes.append(root)

        for _ in range(self.max_iter):
            # Goal biasing


In [33]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

total_df = pd.DataFrame()

for method in classic_method:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
            
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


Map 1
Iteration 1: Time taken: 0.0167 seconds, Success: True
Iteration 2: Time taken: 0.0224 seconds, Success: True
Iteration 3: Time taken: 0.0060 seconds, Success: True
Iteration 4: Time taken: 0.0140 seconds, Success: True
Iteration 5: Time taken: 0.0278 seconds, Success: True
Iteration 6: Time taken: 0.0122 seconds, Success: True
Iteration 7: Time taken: 0.0169 seconds, Success: True
Iteration 8: Time taken: 0.0086 seconds, Success: True
Iteration 9: Time taken: 0.0072 seconds, Success: True
Iteration 10: Time taken: 0.0197 seconds, Success: True
Map 2
Iteration 1: Time taken: 0.1064 seconds, Success: True
Iteration 2: Time taken: 0.2153 seconds, Success: True
Iteration 3: Time taken: 0.1351 seconds, Success: True
Iteration 4: Time taken: 0.1892 seconds, Success: True
Iteration 5: Time taken: 0.1133 seconds, Success: True
Iteration 6: Time taken: 0.1021 seconds, Success: True
Iteration 7: Time taken: 0.0900 seconds, Success: True
Iteration 8: Time taken: 0.0835 seconds, Success: Tr

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
0,RRT,0,1.0,0.015138,248.7,192.016007,0.010318,0.0,-0.000000,-0.000000,-0.000000,0.000000
1,RRT,1,1.0,0.130599,1099.1,320.395966,0.006042,0.0,-0.000000,-0.000000,-0.000000,0.000000
2,RRT,2,1.0,0.030586,499.6,160.435752,0.012195,0.0,-0.000000,-0.000000,-0.000000,0.000000
0,RRT*,0,1.0,0.051288,356.4,171.250880,0.015460,0.0,-238.809167,10.814269,-49.830198,-79.445936
1,RRT*,1,1.0,0.232562,1051.9,232.617256,0.011747,0.0,-78.073377,27.396946,-94.428761,-36.828376
2,RRT*,2,1.0,0.098667,656.2,120.078702,0.022574,0.0,-222.589760,25.154648,-85.101627,-78.766324
0,RRT-Connect,0,1.0,0.005617,82.7,188.456949,0.011243,0.0,62.896568,1.853521,-8.963254,17.447024
1,RRT-Connect,1,1.0,0.013718,231.9,300.184504,0.007368,0.0,89.495811,6.308276,-21.947595,23.720880
2,RRT-Connect,2,1.0,0.012757,211.3,152.873662,0.015972,0.0,58.291941,4.713469,-30.971093,12.236058
0,RRT*-Connect,0,1.0,0.011334,134.2,175.092275,0.016345,0.0,25.127694,8.813709,-58.416539,-2.382258


In [34]:
grouped_avg = total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
BI-RRT,1.0,0.966667,0.004923,126.300000,209.299766,0.010204,-3.333333,86.595593,6.729690,-7.088457,9.240258
BI-RRT*,1.0,0.933333,0.010526,142.266667,172.039175,0.028472,-6.666667,68.738552,21.824934,-210.087100,-50.364201
Improved-RRT*-Connect,1.0,1.000000,1.629583,2580.533333,166.196498,0.015748,0.000000,-4948.382927,25.286808,-69.519727,-1493.361462
Informed-RRT*,1.0,1.000000,4.942214,3958.500000,156.745682,0.028156,0.000000,-17177.996880,29.642142,-196.388301,-5186.748296
RRT,1.0,1.000000,0.058774,615.800000,224.282575,0.009518,0.000000,0.000000,0.000000,0.000000,0.000000
RRT*,1.0,1.000000,0.127506,688.166667,174.648946,0.016593,0.000000,-179.824101,21.121954,-76.453529,-65.013545
RRT*-Connect,1.0,1.000000,0.020937,237.533333,192.077747,0.015952,0.000000,39.637332,13.745243,-68.834073,0.873434
RRT-Connect,1.0,1.000000,0.010697,175.300000,213.838372,0.011528,0.000000,70.228107,4.291755,-20.627314,17.801320


In [35]:
pop_path = "./path_planning/mobj/results/pops/population_generation_9.json"

with open(pop_path, "r") as f:
    data = json.load(f)
    filtered_sorted_algorithms = sorted(
    [alg for alg in data if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
    )

In [36]:
len(filtered_sorted_algorithms)

19

In [37]:
# ref_avg 앞에서 선언됨
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

g_total_df = pd.DataFrame()

for method in filtered_sorted_algorithms:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['objective']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        try:
            planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        except:
            continue
        res, avg_rest = benchmarker.run(planner.plan)
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        g_total_df = pd.concat([g_total_df, res_df], axis=0)

# g_total_df
res1 = g_total_df

Map 1
Iteration 1: Time taken: 0.0139 seconds, Success: True
Iteration 2: Time taken: 0.0045 seconds, Success: True
Iteration 3: Time taken: 0.0025 seconds, Success: True
Iteration 4: Time taken: 0.0036 seconds, Success: True
Iteration 5: Time taken: 0.0035 seconds, Success: True
Iteration 6: Time taken: 0.0062 seconds, Success: True
Iteration 7: Time taken: 0.0023 seconds, Success: True
Iteration 8: Time taken: 0.0071 seconds, Success: True
Iteration 9: Time taken: 0.0030 seconds, Success: True
Iteration 10: Time taken: 0.0034 seconds, Success: True
Map 2
Iteration 1: Time taken: 0.0238 seconds, Success: True
Iteration 2: Time taken: 0.0096 seconds, Success: True
Iteration 3: Time taken: 0.0065 seconds, Success: True
Iteration 4: Time taken: 0.0175 seconds, Success: True
Iteration 5: Time taken: 0.0153 seconds, Success: True
Iteration 6: Time taken: 0.0093 seconds, Success: True
Iteration 7: Time taken: 0.0076 seconds, Success: True
Iteration 8: Time taken: 0.0064 seconds, Success: Tr

In [22]:
whole_df = pd.concat([total_df, g_total_df], axis=0)

In [23]:
grouped_avg = g_total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,success_improvement,time_improvement,length_improvement,objective_score
alg_name,,,,,,,,,
-15.03871,1.0,1.000000,0.008604,159.100000,204.043208,0.000000,76.213594,8.870378,24.638154
-14.13591,1.0,1.000000,0.013600,187.833333,199.432910,0.000000,65.339904,12.138289,22.029629
-11.35589,1.0,1.000000,0.016781,168.566667,200.654919,0.000000,55.041207,10.256080,18.563578
-10.57241,1.0,1.000000,0.010254,152.300000,175.668521,0.000000,72.630647,21.235157,26.036225
-9.55661,1.0,1.000000,0.009573,169.000000,199.482939,0.000000,74.549431,11.184416,24.601712
-9.23875,1.0,1.000000,0.012645,196.033333,209.707866,0.000000,67.748968,6.278594,21.580409
-8.93552,1.0,1.000000,0.015993,216.833333,199.990028,0.000000,56.844661,10.872224,19.227843
-8.86231,1.0,0.966667,0.014326,124.266667,209.325132,-3.333333,61.259962,7.045306,18.120383
-7.55322,1.0,1.000000,0.017287,154.033333,180.891690,0.000000,51.084518,18.307114,18.986778


In [38]:
pop_path = "./path_planning/mobj1/results/pops/population_generation_10.json"

with open(pop_path, "r") as f:
    data = json.load(f)
    filtered_sorted_algorithms = sorted(
    [alg for alg in data if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
    )

len(filtered_sorted_algorithms)

17

In [39]:
# ref_avg 앞에서 선언됨
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

g_total_df = pd.DataFrame()

for method in filtered_sorted_algorithms:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['objective']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        try:
            planner = planning_module.Planner(max_iter=5000, step_size=5.0)
        except:
            continue
        res, avg_rest = benchmarker.run(planner.plan)
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        g_total_df = pd.concat([g_total_df, res_df], axis=0)

res2 = g_total_df

Map 1
Iteration 1: Time taken: 0.0196 seconds, Success: True
Iteration 2: Time taken: 0.0058 seconds, Success: True
Iteration 3: Time taken: 0.0086 seconds, Success: True
Iteration 4: Time taken: 0.0072 seconds, Success: True
Iteration 5: Time taken: 0.0065 seconds, Success: True
Iteration 6: Time taken: 0.0065 seconds, Success: True
Iteration 7: Time taken: 0.0064 seconds, Success: True
Iteration 8: Time taken: 0.0076 seconds, Success: True
Iteration 9: Time taken: 0.0037 seconds, Success: True
Iteration 10: Time taken: 0.0056 seconds, Success: True
Map 2
Iteration 1: Time taken: 0.0188 seconds, Success: True
Iteration 2: Time taken: 0.0178 seconds, Success: True
Iteration 3: Time taken: 0.0138 seconds, Success: True
Iteration 4: Time taken: 0.0108 seconds, Success: True
Iteration 5: Time taken: 0.0125 seconds, Success: True
Iteration 6: Time taken: 0.0119 seconds, Success: True
Iteration 7: Time taken: 0.0082 seconds, Success: True
Iteration 8: Time taken: 0.0113 seconds, Success: Tr

In [40]:
total_df['cls_try'] = 0
res1['cls_try'] = 1
res2['cls_try'] = 2

In [41]:
whole_df = pd.concat([total_df, res1, res2], axis=0)

In [42]:
whole_df

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
0,RRT,0,1.0,0.015138,248.7,192.016007,0.010318,0.0,-0.000000,-0.000000,-0.000000,0.000000,0
1,RRT,1,1.0,0.130599,1099.1,320.395966,0.006042,0.0,-0.000000,-0.000000,-0.000000,0.000000,0
2,RRT,2,1.0,0.030586,499.6,160.435752,0.012195,0.0,-0.000000,-0.000000,-0.000000,0.000000,0
0,RRT*,0,1.0,0.051288,356.4,171.250880,0.015460,0.0,-238.809167,10.814269,-49.830198,-79.445936,0
1,RRT*,1,1.0,0.232562,1051.9,232.617256,0.011747,0.0,-78.073377,27.396946,-94.428761,-36.828376,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,-2.05071,1,1.0,0.016635,213.1,239.538400,0.106945,0.0,87.262714,25.236761,-1670.151864,-302.804206,2
2,-2.05071,2,1.0,0.017740,167.4,124.523413,0.106549,0.0,41.998503,22.384250,-773.685885,-137.660776,2
0,-1.99103,0,1.0,0.008143,77.4,165.910628,0.034140,0.0,46.204465,13.595418,-230.878655,-29.595308,2
1,-1.99103,1,1.0,0.016719,195.7,229.290488,0.103002,0.0,87.198071,28.435276,-1604.886471,-289.130818,2


In [43]:
# alg_name 기준으로 groupby하여 평균값 계산
grouped_avg = whole_df.groupby('alg_name').mean()
aa = pd.DataFrame(grouped_avg)
aa

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
alg_name,,,,,,,,,,,,
-15.03871,1.0,1.000000,0.008717,159.100000,204.043208,0.011024,0.000000,75.780380,8.870378,-15.308153,21.446559,1.0
-14.13591,1.0,1.000000,0.012726,187.833333,199.432910,0.014364,0.000000,68.951808,12.138289,-46.285125,13.856175,1.0
-11.35589,1.0,1.000000,0.018383,168.566667,200.654919,0.012071,0.000000,54.059898,10.256080,-26.085827,13.052020,1.0
-11.12807,1.0,1.000000,0.016015,169.066667,178.872845,0.071576,0.000000,50.953040,19.403655,-678.612633,-116.555884,2.0
-10.57241,1.0,1.000000,0.011115,152.300000,175.668521,0.066166,0.000000,66.898214,21.235157,-621.133706,-99.910246,1.0
-9.55661,1.0,1.000000,0.009738,169.000000,199.482939,0.013761,0.000000,74.313970,11.184416,-41.146114,16.301851,1.0
-9.23875,1.0,1.000000,0.012813,196.033333,209.707866,0.010682,0.000000,68.091916,6.278594,-11.965123,19.290269,1.0
-8.93552,1.0,1.000000,0.016177,216.833333,199.990028,0.012890,0.000000,57.620095,10.872224,-33.941516,12.672170,1.0
-8.86231,1.0,0.966667,0.013333,124.266667,209.325132,0.012041,-3.333333,69.325857,7.045306,-24.201514,0.699849,1.0


In [34]:
col_map = {
    # "map_id": "Map ID",
    "success_rate": "성공률 (Success Rate)",
    "time_avg": "평균 시간 (Time Avg)",
    # "num_nodes_avg": "평균 노드 수 (Num Nodes Avg)",
    "path_length_avg": "평균 경로 길이 (Path Length Avg)",
    "success_improvement": "성공률 개선 (Success Improvement)",
    "time_improvement": "시간 개선 (%)",
    "length_improvement": "경로 개선 (%)",
    "objective_score": "종합 점수 (Objective Score)"
}

df_pretty = grouped_avg.rename(columns=col_map)
df_pretty = df_pretty.round(3)
df_pretty = df_pretty.sort_values("종합 점수 (Objective Score)", ascending=False)
display(df_pretty)

,map_id,성공률 (Success Rate),평균 시간 (Time Avg),num_nodes_avg,평균 경로 길이 (Path Length Avg),성공률 개선 (Success Improvement),시간 개선 (%),경로 개선 (%),종합 점수 (Objective Score)
alg_name,,,,,,,,,
BI-RRT,1.0,0.967,0.004,126.300,209.300,-3.333,88.101,6.730,26.110
RRT-Connect,1.0,1.000,0.012,175.300,213.838,0.000,69.541,4.292,21.721
BI-RRT*,1.0,0.933,0.011,142.267,172.039,-6.667,68.539,21.825,21.593
15.44831,1.0,1.000,0.013,147.633,180.308,0.000,58.762,18.807,21.390
RRT*-Connect,1.0,1.000,0.020,237.533,192.078,0.000,40.091,13.745,14.776
RRT,1.0,1.000,0.055,615.800,224.283,0.000,0.000,0.000,0.000
RRT*,1.0,1.000,0.127,688.167,174.649,0.000,-192.159,21.122,-53.423
623.91658,1.0,1.000,0.141,703.067,217.119,0.000,-252.731,3.369,-75.146
2116.54822,1.0,0.900,0.150,645.033,166.335,-10.000,-271.184,24.860,-81.383


### LM TEST

In [1]:
from eoh.llm.interface_LLM import InterfaceLLM
from eoh.utils.getParas import Paras
from eoh.methods.eoh.eoh_evolution import Evolution
from eoh.problems.optimization.classic_benchmark_path_planning.utils.prompts import GetPrompts
import re
paras = Paras()

In [3]:
ee = Evolution("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True, GetPrompts())
illm = InterfaceLLM("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True)

- check LLM API
remote llm api is used ...
- check LLM API
remote llm api is used ...


In [176]:
pc = ee.get_prompt([result_data[3]], 'e1')

In [8]:
gcode = illm.get_response(pc)

In [9]:
print(gcode)

{This algorithm implements a Bidirectional Informed RRT* planner with heuristic-based informed sampling and path smoothing. It grows two trees from start and goal states, focusing samples in an ellipsoidal informed set to concentrate the search on promising regions, improving efficiency and solution quality. It applies rewiring to optimize paths and uses a post-processing smoothing step for path smoothness and shorter paths.}  
(A random sample is drawn within an ellipsoid defined by the current best path cost to bias sampling, then nodes are extended towards samples from both trees. The two trees attempt to connect after each iteration, enabling quick discovery of a path. Rewiring improves local paths iteratively. When a path is found, path smoothing refines it by shortcutting unnecessary waypoints.)

```python
from typing import List, Tuple, NamedTuple, Optional
from math import dist, sqrt, cos, sin, acos
import random

# --- Node class ---
class Node:
    def __init__(self, position

In [11]:
c, m, a = ee._extract_alg(gcode)

In [18]:
response = ee.get_analysis(classic_method[0]['code'], classic_method[5]['code'], 'time')

In [17]:
classic_method[0]

{'algorithm': 'RRT',
 'algorithm_description': 'This algorithm is a sampling-based path planning algorithm that incrementally builds a space-filling tree rooted at the start position by randomly sampling the configuration space and extending the nearest existing node toward the sample. It continues this process until the goal is reached or a maximum number of iterations is exceeded.',
 'planning_mechanism': 'The planner randomly samples free configurations, finds the nearest node in the current tree, attempts to extend toward the sample by a fixed step size, and adds the new node if the move is valid. This repeats until the goal is reached or iteration limit is hit.',
 'code': '\nclass Node:\n    def __init__(self, position, parent=None, cost=0.0):\n        self.position = position\n        self.parent = parent\n        self.cost = cost\n        self.children = []\n        self.valid = True\n\n    def add_child(self, child_node):\n        self.children.append(child_node)\n\n\nclass Pla

In [19]:
print(response)

1. Summary of key changes:  
   - alg1 implements a bidirectional search with two trees grown from start and goal, while alg2 uses a unidirectional single tree.  
   - alg1 includes rewiring to optimize paths and maintain cost-minimizing parent-child relationships; alg2 lacks rewiring and cost optimization beyond simple parent assignment.  
   - alg2 applies goal biasing (goal_sample_rate) to probabilistically sample the goal position, increasing the chances of connecting to the goal.  
   - alg2 uses a smaller step_size (3.0 vs 5.0) potentially providing finer resolution in path expansion.  
   - alg1 maintains and updates children lists and edges extensively during rewiring; alg2 manages simpler parent-child linkage without edge rewiring.

2. Primary contributors to the performance improvement:  
   - Introduction of goal biasing in sampling significantly speeds goal reachability.  
   - Simpler unidirectional search reduces computational overhead compared to managing two trees and r